In [ ]:
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink


##Downloading and Running An LLM

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [ ]:
# we can directly use model and tokenizer directly from pipeline object wrapper
# here we have downloaded model on local machine and then using it.

from transformers import pipeline

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=False,
    return_full_text=False,
    )

tokenizer = AutoTokenizer.from_pretrained('microsoft/Phi-3-mini-4k-instruct')

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
# The prompt (user input / query)
messages = [
    {"role": "user", "content": "Create a funny joke about chickens."}
]

# Generate output
output = generator(messages)
print(output[0]["generated_text"])

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


 Why did the chicken join the band? Because it had the drumsticks!


In [ ]:
prompt = "Write an email apologizing her for her loss of here brother in covid pandemic .Explain how it happened. <|assistant|>"

# tokenize the i/p
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda')

# generate the text
generation_output = model.generate(
    input_ids = input_ids,
    max_new_tokens = 30
)

print(tokenizer.decode(generation_output[0]))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Write an email apologizing her for her loss of here brother in covid pandemic .Explain how it happened. <|assistant|> Subject: My Deepest Apologies for the Loss of Your Brother

Dear [Recipient's Name],




In [ ]:
print(input_ids)

tensor([[14350,   385,  4876, 27746,  5281,   902,   363,   902,  6410,   310,
          1244,  8099,   297, 18838,   333,  7243, 24552,   869,  9544,  7420,
           920,   372,  9559, 29889, 29871, 32001]], device='cuda:0')


In [ ]:
for id in input_ids[0]:
  print(tokenizer.decode(id))

Write
an
email
apolog
izing
her
for
her
loss
of
here
brother
in
cov
id
pan
demic
.
Exp
lain
how
it
happened
.

<|assistant|>


In [ ]:
generation_output

tensor([[14350,   385,  4876, 27746,  5281,   902,   363,   902,  6410,   310,
          1244,  8099,   297, 18838,   333,  7243, 24552,   869,  9544,  7420,
           920,   372,  9559, 29889, 29871, 32001,  3323,   622, 29901,  1619,
         21784,   342,  6225, 11763,   363,   278,   365,  2209,   310,  3575,
          4358,   721,    13,    13, 29928,   799,   518,  1123,  7334,   993,
         29915, 29879,  4408,  1402,    13,    13]], device='cuda:0')

In [ ]:
print(tokenizer.decode(385))
print(tokenizer.decode(6225))
print(tokenizer.decode(9544))
print(tokenizer.decode(13))
print(tokenizer.decode(29871))
print(tokenizer.decode(29901))
print(tokenizer.decode(920))

an
Ap
Exp



:
how


#**Comparing Trained LLM Tokenizers**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
  tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
  token_ids = tokenizer(sentence).input_ids
  for idx, t in enumerate(token_ids):
    print(
        f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
    )

In [ ]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

In [ ]:
# The official is `tiktoken` but this the same tokenizer on the HF platform

In [ ]:
show_tokens(text, "bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

In [ ]:
show_tokens(text,"bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

In [ ]:
show_tokens(text, "gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

In [ ]:
show_tokens(text, "google/flan-t5-small")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600  </s> 

In [ ]:
show_tokens(text, "Xenova/gpt-4")

tokenizer_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 12 . 0 * 50 = 600 
 

In [ ]:
# Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.

In [ ]:
show_tokens(text, "bigcode/starcoder2-15b")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [ ]:
show_tokens(text, "facebook/galactica-1.3b")

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]


 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : "      "  Three  t abs :   "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [ ]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

 
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :"    " Three tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [ ]:
show_tokens(text, "distilgpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

In [ ]:
import pprint

In [ ]:
def compare_tokenizers(model_names):
  info={}
  for name in model_names:
    tokenizer = AutoTokenizer.from_pretrained(name)
    info[name]={
        "vocab_size": tokenizer.vocab_size,
        "full_vocab": len(tokenizer),
        "algo": tokenizer.__class__.__name__,
        "special_tokens": tokenizer.all_special_tokens,
    }
  return info


if __name__=="__main__":
  models = [
        "bert-base-cased",
        "bert-base-uncased",
        "gpt2",
        "distilgpt2",
        "microsoft/Phi-3-mini-4k-instruct",
        "google/flan-t5-small",
        "xenova/gpt-4",
        "bigcode/starcoder2-15b",
        "facebook/galactica-1.3b",
    ]
  pprint.pprint(compare_tokenizers(models))

tokenizer_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

{'bert-base-cased': {'algo': 'BertTokenizerFast',
                     'full_vocab': 28996,
                     'special_tokens': ['[UNK]',
                                        '[SEP]',
                                        '[PAD]',
                                        '[CLS]',
                                        '[MASK]'],
                     'vocab_size': 28996},
 'bert-base-uncased': {'algo': 'BertTokenizerFast',
                       'full_vocab': 30522,
                       'special_tokens': ['[UNK]',
                                          '[SEP]',
                                          '[PAD]',
                                          '[CLS]',
                                          '[MASK]'],
                       'vocab_size': 30522},
 'bigcode/starcoder2-15b': {'algo': 'GPT2TokenizerFast',
                            'full_vocab': 49152,
                            'special_tokens': ['<|endoftext|>',
                                               '<f

In [ ]:
text = "def my_function(param): return param*2  # test code"
for name, tok in compare_tokenizers(models).items():
    ids = AutoTokenizer.from_pretrained(name)(text).input_ids
    print(name, "→", len(ids), "tokens")


bert-base-cased → 19 tokens
bert-base-uncased → 19 tokens
gpt2 → 15 tokens
distilgpt2 → 15 tokens
microsoft/Phi-3-mini-4k-instruct → 15 tokens
google/flan-t5-small → 19 tokens
xenova/gpt-4 → 13 tokens
bigcode/starcoder2-15b → 15 tokens
facebook/galactica-1.3b → 16 tokens


##**Less tokens = higher efficiency, but fewer tokens might mean poorer granularity**

In [ ]:
# Vocabulary size = number of unique tokens a tokenizer knows.
# Token count for a sentence = how many tokens a sentence is split into.

# Fewer tokens per sentence usually comes from a larger vocab (more whole words).

# Smaller vocab = more subwords = more tokens per sentence.

# But the type of tokenizer (BPE, WordPiece, SentencePiece, etc.) affects this too.



In [ ]:
'''
BERT tokenizers (cased vs. uncased): Preprocess for masked language modeling; WordPiece segmentation. Case sensitivity affects vocabulary size marginally (30522 uncased vs ~30k cased).

GPT‑2 / DistilGPT‑2: Auto-regressive generation using byte-level BPE, handles rare characters robustly. DistilGPT‑2 is lighter but uses the same tokenizer.

Phi‑3‑mini‑4k‑instruct: Instruction‑tuned conversational model with 4K context; uses SentencePiece plus added special tokens.

FLAN‑T5‑small: Encoder‑decoder model pre‑trained for instruction tasks; SentencePiece vocabulary (~32k + extras).

xenova/gpt‑4: A reimplementation of GPT‑4's tokenizer for HF, based on OpenAI’s tiktoken; optimized for multilingual and efficiency.

StarCoder2‑15B: Code-generation model; Byte-level BPE with code-aware tokens, long context windows.

Galactica‑1.3B: Sci‑text specialized; vocabulary learned from scientific corpus; tokenizer stored as ~2 MB JSON.
'''

In [ ]:
'''
✅ Which is "best"?
Use‑case matters most:

-Natural language general → GPT‑2 or GPT‑4 tokenizers offer broader coverage with robust handling.
-Instructional text → FLAN‑T5 or Phi‑3 include special tokens for prompt patterns.
-Scientific text → Galactica’s tokenizer aligns with domain-specific terms.
-Code generation → StarCoder2 has code-optimized tokens.

Tokenizer algorithm:
Byte-level BPE (GPT‑2, StarCoder2): Great OOV handling and multilingual robustness.
SentencePiece / WordPiece (T5, BERT, Phi‑3): Efficient and structured but may struggle with rare scripts.

'''

'''
BERT (WordPiece, vocab ~30K)  bcz it has smaller vocab and uses subwords it splits unknown or rare words into muiltiple tokens
MORE TOKENS -> SMALLER VOCAB

GPT-2 (Byte-level BPE, vocab 50K)might have this whole word as one token (or two max), because its vocab is trained on bytes and includes very frequent whole words and even emojis.
FEWER TOKENS - > LARGER VOCAB



'''

'\n✅ Which is "best"?\nUse‑case matters most:\n\nNatural language general → GPT‑2 or GPT‑4 tokenizers offer broader coverage with robust handling.\n\nInstructional text → FLAN‑T5 or Phi‑3 include special tokens for prompt patterns.\n\nScientific text → Galactica’s tokenizer aligns with domain-specific terms.\n\nCode generation → StarCoder2 has code-optimized tokens.\n\nTokenizer algorithm:\n\nByte-level BPE (GPT‑2, StarCoder2): Great OOV handling and multilingual robustness.\n\nSentencePiece / WordPiece (T5, BERT, Phi‑3): Efficient and structured but may struggle with rare scripts.\n\n'